In [1]:
!pip install opencv-python


  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl (5.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.0.2 which is incompatible.
You should consider upgrading via the '/Users/ankitkumar/Downloads/mini project.py/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install mediapipe


  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
You should consider upgrading via the '/Users/ankitkumar/Downloads/mini project.py/venv/bin/python3 -m pip install --upgrade pip' command.


In [3]:
pip install --force-reinstall mediapipe


  Using cached mediapipe-0.10.21-cp39-cp39-macosx_11_0_universal2.whl (49.2 MB)
  Using cached jaxlib-0.4.30-cp39-cp39-macosx_11_0_arm64.whl (66.7 MB)
  Using cached sentencepiece-0.2.1-cp39-cp39-macosx_11_0_arm64.whl (1.3 MB)
  Using cached jax-0.4.30-py3-none-any.whl (2.0 MB)
  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl (46.9 MB)
  Using cached absl_py-2.3.1-py3-none-any.whl (135 kB)
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)
  Using cached flatbuffers-25.9.23-py2.py3-none-any.whl (30 kB)
  Using cached attrs-25.4.0-py3-none-any.whl (67 kB)
  Using cached protobuf-4.25.8-cp37-abi3-macosx_10_9_universal2.whl (394 kB)
  Using cached matplotlib-3.9.4-cp39-cp39-macosx_11_0_arm64.whl (7.8 MB)
  Using cached sounddevice-0.5.3-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl (108 kB)
  Using cached cffi-2.0.0-cp39-cp39-macosx_11_0_arm64.whl (180 kB)
  Using cached pycparser-2.23-py3-none-any.whl (118 kB)
  Using cached ml_dt

In [4]:
pip install "numpy<2.0"

You should consider upgrading via the '/Users/ankitkumar/Downloads/mini project.py/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install pyautogui


You should consider upgrading via the '/Users/ankitkumar/Downloads/mini project.py/venv/bin/python3 -m pip install --upgrade pip' command.


In [6]:
!pip install pynput


You should consider upgrading via the '/Users/ankitkumar/Downloads/mini project.py/venv/bin/python3 -m pip install --upgrade pip' command.


In [7]:
import cv2
import mediapipe as mp
import util
import pyautogui
import random
from pynput.mouse import Button, Controller
mouse = Controller()

In [8]:
screen_width, screen_height = pyautogui.size()
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1
)

I0000 00:00:1764629440.719595  625766 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1764629440.732133  628187 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764629440.737658  628191 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [9]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]  
        index_finger_tip = hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
        return index_finger_tip
    return None, None

In [10]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y / 2 * screen_height)
        pyautogui.moveTo(x, y)

In [11]:
def is_left_click(landmark_list, thumb_index_dist):
    return (
            util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) > 90 and
            thumb_index_dist > 50
    )

In [12]:
def is_right_click(landmark_list, thumb_index_dist):
    return (
            util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
            util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 90  and
            thumb_index_dist > 50
    )

In [13]:
def is_double_click(landmark_list, thumb_index_dist):
    return (
            util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
            thumb_index_dist > 50
    )

In [14]:
def is_screenshot(landmark_list, thumb_index_dist):
    return (
            util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
            thumb_index_dist < 50
    )

In [15]:
def detect_gesture(frame, landmark_list, processed):
    if len(landmark_list) >= 21:

        index_finger_tip = find_finger_tip(processed)
        thumb_index_dist = util.get_distance([landmark_list[4], landmark_list[5]])

        if util.get_distance([landmark_list[4], landmark_list[5]]) < 50  and util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 90:
            move_mouse(index_finger_tip)
        elif is_left_click(landmark_list,  thumb_index_dist):
            mouse.press(Button.left)
            mouse.release(Button.left)
            cv2.putText(frame, "Left Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif is_right_click(landmark_list, thumb_index_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame, "Right Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        elif is_double_click(landmark_list, thumb_index_dist):
            pyautogui.doubleClick()
            cv2.putText(frame, "Double Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        elif is_screenshot(landmark_list,thumb_index_dist ):
            im1 = pyautogui.screenshot()
            label = random.randint(1, 1000)
            im1.save(f'my_screenshot_{label}.png')
            cv2.putText(frame, "Screenshot Taken", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)



In [ ]:
def main():
    cap = cv2.VideoCapture(0)
    draw = mp.solutions.drawing_utils

    try:
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            frame = cv2.flip(frame, 1)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            processed = hands.process(frameRGB)

            landmark_list = []

            if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]

                draw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)

                for lm in hand_landmarks.landmark:
                    landmark_list.append((lm.x, lm.y))

                print(landmark_list)
                
            detect_gesture(frame, landmark_list, processed)
            
            cv2.imshow('Frame', frame)


            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()


if __name__ == '__main__':
    main()


W0000 00:00:1764629465.163843  628192 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


[(0.7415494322776794, 1.0747085809707642), (0.7940333485603333, 1.025797963142395), (0.8213081359863281, 0.9035934209823608), (0.8250422477722168, 0.7989940643310547), (0.8288233280181885, 0.7175068259239197), (0.7862399816513062, 0.8294587731361389), (0.7862822413444519, 0.7120487093925476), (0.7850559949874878, 0.6291457414627075), (0.7837631702423096, 0.5642420053482056), (0.7549729943275452, 0.8235399723052979), (0.7554539442062378, 0.6935548186302185), (0.7543675899505615, 0.5994411706924438), (0.7520981431007385, 0.5204161405563354), (0.7234658598899841, 0.8371116518974304), (0.723274290561676, 0.7138984799385071), (0.7243273258209229, 0.62119060754776), (0.725421130657196, 0.5416514873504639), (0.6891909837722778, 0.8692425489425659), (0.6785395741462708, 0.7710369229316711), (0.6754275560379028, 0.6975950598716736), (0.6762702465057373, 0.6337437033653259)]
[(0.7709405422210693, 1.0010064840316772), (0.707445502281189, 0.9499805569648743), (0.6612316370010376, 0.867026090621948